# AFEW Preprocessing

## Make Frames

In [5]:
import os
from tqdm import tqdm

In [6]:
DATA_DIR=r'D:\Users\amira\Documents\datasets\emotions\AudioVideo'

In [ ]:
def process_dir(dirname):
    resdir=os.path.join(DATA_DIR,'frames', 'Val_AFEW')
    d = os.path.normpath(os.path.join(DATA_DIR, 'Val_AFEW', dirname))
    for filename in tqdm(os.listdir(d)):
        if os.path.isdir(os.path.normpath(os.path.join(d,filename))):
            videofile=None
            for fn in os.listdir(os.path.normpath(os.path.join(d,filename))):
                videofile=fn
            if videofile is None:
                continue
            filename=os.path.normpath(os.path.join(filename,videofile))
        fn, ext = os.path.splitext(os.path.basename(filename))
        outdir=os.path.normpath(os.path.join(resdir,dirname,fn))
        if not os.path.exists(outdir):
            os.makedirs(outdir)
        command = "ffmpeg -r 1 -i "+os.path.join(d,filename) + " -r 1 "+outdir+"/%05d.png"
        command = os.path.normpath(command)
        print(command)
        os.system(command=command)


for emotion_name in os.listdir(os.path.normpath(os.path.join(DATA_DIR, 'Val_AFEW'))):
    process_dir(emotion_name)

## Get one face from frame

In [1]:
import os
from PIL import Image
import cv2

from collections import defaultdict
import os
import numpy as np
from tqdm import tqdm
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
from facial_analysis import FacialImageProcessing
imgProcessing=FacialImageProcessing(False)

{'D:\\Users\\amira\\emotion-recognition\\code\\mtcnn.pb': ''}


In [8]:
DATA_DIR='D:/Users/amira/Documents/datasets/emotions/AudioVideo'

In [ ]:
INPUT_SIZE = (224,224)
def save_faces(source_path,save_path):
    if not os.path.exists(save_path):
        os.makedirs(save_path)
    for folder in tqdm(os.listdir(source_path)):
        #print(folder)
        if not os.path.exists(os.path.join(save_path, folder)):
            os.mkdir(os.path.join(save_path, folder))

            for image in os.listdir(os.path.join(source_path, folder)):
                filename = os.path.join(source_path, folder, image)
                frame_bgr = cv2.imread(filename)
                frame = cv2.cvtColor(frame_bgr, cv2.COLOR_BGR2RGB)
                bounding_boxes, _ = imgProcessing.detect_faces(frame)

                if len(bounding_boxes)==0:
                    print('No faces found for ',filename)
                    face_img = frame_bgr
                    faceFound='noface'
                else:
                    if len(bounding_boxes)>1:
                        print('Too many faces (',len(bounding_boxes),') found for ',filename)
                        bounding_boxes=bounding_boxes[:1]

                    b=[int(bi) for bi in bounding_boxes[0]]
                    x1,y1,x2,y2=b[0:4]
                    face_img=frame_bgr[y1:y2,x1:x2,:]

                    if np.prod(face_img.shape)==0:
                        print('Empty face ',b,' found for ',filename)
                        continue
                
                    faceFound=''

                root,ext=os.path.splitext(image)
                cv2.imwrite(os.path.join(save_path, folder, root+faceFound+ext), face_img) 
        else:
            print(folder)

for emotion_name in os.listdir(os.path.normpath(os.path.join(DATA_DIR, 'frames/Train_AFEW'))):        
    save_faces(os.path.join(DATA_DIR,'frames/Train_AFEW', emotion_name), os.path.join(DATA_DIR,'faces/Train_AFEW', emotion_name))

## Get multiple faces from frame

In [1]:
import os
from PIL import Image
import cv2

from collections import defaultdict
import os
import numpy as np
from tqdm import tqdm
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
from facial_analysis import FacialImageProcessing
imgProcessing=FacialImageProcessing(False)

{'D:\\Users\\amira\\emotion-recognition\\code\\mtcnn.pb': ''}


In [3]:
DATA_DIR='D:/Users/amira/Documents/datasets/emotions/AudioVideo'

In [ ]:
INPUT_SIZE = (224,224)
def save_faces(source_path,save_path):
    if not os.path.exists(save_path):
        os.makedirs(save_path)
    for folder in tqdm(os.listdir(source_path)):
        #print(folder)
        if not os.path.exists(os.path.join(save_path, folder)):
            os.mkdir(os.path.join(save_path, folder))

            for image in os.listdir(os.path.join(source_path, folder)):
                filename = os.path.join(source_path, folder, image)
                frame_bgr = cv2.imread(filename)
                frame = cv2.cvtColor(frame_bgr, cv2.COLOR_BGR2RGB)
                bounding_boxes, _ = imgProcessing.detect_faces(frame)

                if len(bounding_boxes)==0:
                    print('No faces found for ',filename)
                    face_img = frame_bgr
                    faceFound='noface'
                else:
                    print('(',len(bounding_boxes),') faces found for ',filename)
                    for i in range(len(bounding_boxes)):
                        bounding_box=bounding_boxes[i]

                        b=[int(bi) for bi in bounding_box]
                        x1,y1,x2,y2=b[0:4]
                        face_img=frame_bgr[y1:y2,x1:x2,:]

                        if np.prod(face_img.shape)==0:
                            print('Empty face ',b,' found for ',filename)
                            continue
                    
                        faceFound='_'+str(i)

                #face_img=cv2.resize(face_img,INPUT_SIZE)
                root,ext=os.path.splitext(image)
                cv2.imwrite(os.path.join(save_path, folder, root+faceFound+ext), face_img) 
        else:
            print(folder)

for emotion_name in os.listdir(os.path.normpath(os.path.join(DATA_DIR, 'frames/Train_AFEW'))):        
    save_faces(os.path.join(DATA_DIR,'frames/Train_AFEW', emotion_name), os.path.join(DATA_DIR,'faces/Train_AFEW', emotion_name))

## Get OpenFace features

In [1]:
import os
from tqdm import tqdm

In [2]:
DATA_DIR='D:/Users/amira/Documents/datasets/emotions/AudioVideo'

In [18]:
def extract_openface_features(dirname,outdir):
    print(dirname)
    if not os.path.exists(outdir):
        os.makedirs(outdir)
    for emotion_name in os.listdir(dirname):
        for video_name in tqdm(os.listdir(os.path.join(dirname,emotion_name))):
            command=r'D:\Users\amira\Documents\OpenFace_2.2.0_win_x64\FeatureExtraction.exe -pose -aus -gaze -out_dir '+outdir+emotion_name
            frames_dir=os.path.join(dirname,emotion_name,video_name)
            if os.path.isdir(frames_dir):
                command+=' -fdir '+frames_dir
                os.system(command=command)
    

cur_dir=os.getcwd()
extract_openface_features(os.path.join(DATA_DIR,'frames/Train_AFEW'),os.path.join(DATA_DIR,'openface/Train_AFEW/'))
extract_openface_features(os.path.join(DATA_DIR,'frames/Val_AFEW'),os.path.join(DATA_DIR,'openface/Val_AFEW/'))
os.chdir(cur_dir)

D:/Users/amira/Documents/datasets/emotions/AudioVideo\frames/Train_AFEW


100%|██████████| 74/74 [02:55<00:00,  2.38s/it]


D:/Users/amira/Documents/datasets/emotions/AudioVideo\frames/Val_AFEW


100%|██████████| 46/46 [01:31<00:00,  1.98s/it]
